# M09 Homework
- Name: Sam Remmey
- Net ID: sqr8ap
- URL of this file in GitHub: 

In [8]:
import pandas as pd
import numpy as np
from gensim.models import word2vec
from gensim.corpora import Dictionary
from sklearn.manifold import TSNE as tsne
import plotly_express as px

In [10]:
import gensim
gensim.__version__

'4.3.0'

In [2]:
import configparser
config = configparser.ConfigParser()
config.read("../../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']

In [18]:
data_prefix = 'austen-melville'
table_dir =  f'{output_dir}'
OHCO = ['book_id', 'chap_id', 'para_num', 'sent_num', 'token_num']
PARA = OHCO[:4] # Paragraphs
SENT = OHCO[:5] # Sentences
BAG = PARA

In [6]:
# word2vec parameters
w2v_params = dict(
    window = 5,
    vector_size = 246,
    min_count = 50, # THIS LIMITS OUR VOCAB
    workers = 4
)

In [44]:
TOKENS = pd.read_csv(f'{table_dir}/{data_prefix}-CORPUS.csv').set_index(OHCO)
TOKENS = TOKENS[TOKENS['pos'].isin(['NNS','NN','VB','VBD','VBN','VBG','VBP','VBZ'])]

LIB = pd.read_csv(f'{table_dir}/{data_prefix}-LIB.csv')

In [36]:
TOKENS.sample()

,,,,,pos_tuple,pos,token_str,term_str,pos_group
book_id,chap_id,para_num,sent_num,token_num,,,,,
946,25,3,24,15,"('am', 'VBP')",VBP,am,am,VB


In [38]:
LIB.sample()

,source_file_path,author,title,chap_regex,book_len,n_chaps
book_id,,,,,,
1342,/Users/Samantha/Desktop/MSDS/DS5001/data/guten...,"AUSTEN, JANE",PRIDE AND PREJUDICE,^Chapter\s+\d+$,122126,61


In [50]:
# Join TOKENS to LIB on book_id, then separate by author

both = TOKENS.reset_index().merge(LIB[['book_id','author']], on='book_id').set_index(TOKENS.index.names)

TOKENS_austen = both[both['author'] == 'AUSTEN, JANE']
TOKENS_melville = both[both['author'] == 'MELVILLE, HERMAN']

In [56]:
len(both) == len(TOKENS_austen) + len(TOKENS_melville)

True

In [60]:
# Convert to gensim corpora

docs_austen = TOKENS_austen.dropna(subset=['term_str'])\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()
docs_austen = [doc for doc in docs_austen if len(doc) > 1]

docs_melville = TOKENS_melville.dropna(subset=['term_str'])\
    .groupby(BAG)\
    .term_str.apply(lambda  x:  x.tolist())\
    .reset_index()['term_str'].tolist()
docs_melville = [doc for doc in docs_melville if len(doc) > 1]

In [66]:
# Extract vocab tables

